In [ ]:
!pip uninstall googletrans
!pip install googletrans==3.1.0a0


In [ ]:
!pip install google_trans_new

In [ ]:
import pandas as pd
import numpy as np
from googletrans import Translator
from google_trans_new import google_translator  
import os
import re

In [ ]:
train_path = '/content/drive/MyDrive/mediaeval2016/train-ekphrasis.csv'
dev_path = '/content/drive/MyDrive/mediaeval2016/dev-ekphrasis.csv'
test_path = '/content/drive/MyDrive/mediaeval2016/test-ekphrasis.csv'

In [ ]:
train = pd.read_csv(train_path)
train.info()
display(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12376 entries, 0 to 12375
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_id       12376 non-null  int64 
 1   post_text     12376 non-null  object
 2   user_id       12376 non-null  int64 
 3   image_id(s)   12376 non-null  object
 4   username      12376 non-null  object
 5   timestamp     12376 non-null  object
 6   label         12376 non-null  object
 7   cleaned_text  12376 non-null  object
 8   enc_label     12376 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 870.3+ KB


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,264364835565883393,"Dear mainstream media, Sandy fucked us up too....",366606850,sandyB_real_10,NevenaBK,Fri Nov 02 13:54:23 +0000 2012,real,"Dear mainstream media , Sandy fucked us up too...",0
1,262962084801814528,Crazy pic taken in #NewYork today! #Hurricane ...,47332641,sandyA_fake_29,iChrisHarrison,Mon Oct 29 17:00:21 +0000 2012,fake,Crazy pic taken in #NewYork today ! #Hurricane...,1
2,264415086981038080,Hurricane A-rod #mlbmemes http://t.co/1cyx4OdM,63635369,sandyB_real_10,wiegman_tommy,Fri Nov 02 17:14:03 +0000 2012,real,Hurricane A - rod #mlbmemes <url>,0
3,591987644134887425,Nepal's historic Dharahara Tower collapses in ...,485782675,nepal_25,EliseArja,Sat Apr 25 15:30:27 +0000 2015,real,Nepal ' s historic Dharahara Tower collapses i...,0
4,263997194540032003,O furacão Sandy passou por Nova Iorque e destr...,354373541,sandyA_real_48,GALOUCURACORINT,Thu Nov 01 13:33:30 +0000 2012,real,Hurricane Sandy passed through New York and de...,0


In [ ]:
dev = pd.read_csv(dev_path)
dev.info()
display(dev.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   post_id       1386 non-null   int64 
 1   post_text     1386 non-null   object
 2   user_id       1386 non-null   int64 
 3   image_id(s)   1386 non-null   object
 4   username      1386 non-null   object
 5   timestamp     1386 non-null   object
 6   label         1386 non-null   object
 7   cleaned_text  1386 non-null   object
 8   enc_label     1386 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 97.6+ KB


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263067887546281984,"Hurricane #Sandy, by @time http://t.co/ixAeykg1",112749098,sandyA_real_22,amelieamaral,Tue Oct 30 00:00:46 +0000 2012,real,"Hurricane #Sandy , by <user> <url>",0
1,263012006943158274,That shit cray.. #sandy http://t.co/PmI0RKvq,322326492,sandyA_fake_29,HoZe_B1,Mon Oct 29 20:18:43 +0000 2012,fake,That shit cray . . #sandy <url>,1
2,263218608790970369,The effects of hurricane sandy in Harlem New Y...,30071828,sandyA_real_20,AKidNameTraveee,Tue Oct 30 09:59:41 +0000 2012,real,The effects of hurricane sandy in Harlem New Y...,0
3,263098784236597248,O.O #Jesuuuu RT @Famosos_RD: Este tiburon no s...,180451067,sandyA_fake_32,edluis_11,Tue Oct 30 02:03:32 +0000 2012,fake,O . O #Jesuuuu RT <user> : Este tiburon no se ...,1
4,264017358262243328,"\""@TuiterHits: IMPRESIONANTE: Las inundaciones...",419835315,sandyA_fake_12,elenitacanon15,Thu Nov 01 14:53:38 +0000 2012,fake,"\ "" <user> : IMPRESIONANTE : Las inundaciones ...",1


In [ ]:
test = pd.read_csv(test_path)
test.info()
display(test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1117 entries, 0 to 1116
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   post_id       1117 non-null   int64  
 1   post_text     1117 non-null   object 
 2   user_id       1117 non-null   float64
 3   username      1117 non-null   object 
 4   image_id      1117 non-null   object 
 5   timestamp     1117 non-null   object 
 6   label         1117 non-null   object 
 7   cleaned_text  1117 non-null   object 
 8   enc_label     1117 non-null   int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 78.7+ KB


,post_id,post_text,user_id,username,image_id,timestamp,label,cleaned_text,enc_label
0,665333038944002048,"Tristesse...😢🙏 \n#Bataclan sold out, musiciens...",5.547267e+08,Louise_Officiel,attacks_paris_1,Sat Nov 14 00:58:52 +0000 2015,fake,"Tristesse . . . 😢 🙏 #Bataclan sold out , music...",1
1,665324167785410560,RT @Proyecto40: #ÚltimaHora Espectacular fotog...,2.827010e+09,MarinoCarril,attacks_paris_1,Sat Nov 14 00:23:37 +0000 2015,fake,<user> : #ÚltimaHora Espectacular fotografía s...,1
2,665333370205765632,RT @Javivi1976: #Bataclan esta noche antes de ...,3.776037e+08,MireiaMaroto,attacks_paris_1,Sat Nov 14 01:00:11 +0000 2015,fake,<user> : #Bataclan esta noche antes de los ate...,1
3,665326188735295489,RT @Pizzigatas: El hombre tiene que establecer...,1.102841e+08,maria_cabrera,attacks_paris_1,Sat Nov 14 00:31:39 +0000 2015,fake,<user> : El hombre tiene que establecer un fin...,1
4,665389341141659648,🇫🇷 #Paris https://t.co/zjjRPC7USm,1.653118e+07,RaycMolina,attacks_paris_1,Sat Nov 14 04:42:35 +0000 2015,fake,🇫 🇷 #Paris <url>,1


# Pre-processing

In [ ]:
!pip install ekphrasis

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import preprocessor as p


In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'user'],
    fix_html=True,  # fix HTML tokens
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons],
    mode='fast'
)


In [ ]:
translator = google_translator()  

In [ ]:
def preprocessing(text):
  p.set_options(p.OPT.RESERVED) #Delete RT
  text = p.clean(text)

  text = re.sub(r'\n', ' ', text)
  text = re.sub(r'\\n', ' ', text)
  text = re.sub(r'(&amp;|&gt;|&lt;)', " ", text) #Delete some special words
  text = " ".join(text_processor.pre_process_doc(text))
  text = re.sub(r"\s+", " ", text)
  
  return text  

In [ ]:
translator.translate(preprocessing('RT @alien_vq: الفنانة السويسرية ميلو مواريه تتعرى إحتجاجا على التحرش في ألمانيا\\n"احترمونا،نحن لسنا لعبة مجانية حتى ولو كنا عراة'),lang_tgt='en')

'<user>: Swiss artist Milo Moire is undressed in protest against harassment in Germany "Respect us, we are not a free game even if we are naked '

In [ ]:
import tqdm

# train['cleaned_text'] = train['post_text']
# dev['cleaned_text'] = dev['post_text']
# test['cleaned_text'] = test['post_text']

for i in tqdm.tqdm(range(len(train['post_text']))):
  sent = train['cleaned_text'][i]
  # sent = preprocessing(sent)
  train['cleaned_text'][i] = translator.translate(sent,lang_tgt='en')

for i in tqdm.tqdm(range(len(dev['post_text']))):
  sent = dev['cleaned_text'][i]
  # sent = preprocessing(sent)
  dev['cleaned_text'][i] = translator.translate(sent,lang_tgt='en')

for i in tqdm.tqdm(range(len(test['post_text']))):
  sent = test['cleaned_text'][i]
  # sent = preprocessing(sent)
  test['cleaned_text'][i] = translator.translate(sent,lang_tgt='en')


  0%|          | 0/12376 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
  0%|          | 0/1386 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
  0%|          | 0/1117 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [ ]:
train['enc_label'] = train['label']
train['enc_label'] = train['enc_label'].apply(lambda x: 1 if x=='fake' else 0)
dev['enc_label'] = dev['label']
dev['enc_label'] = dev['enc_label'].apply(lambda x: 1 if x=='fake' else 0)
test['enc_label'] = test['label']
test['enc_label'] = test['enc_label'].apply(lambda x: 1 if x=='fake' else 0)

In [ ]:
display(train.head()), display(dev.head()), display(test.head())

,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,264364835565883393,"Dear mainstream media, Sandy fucked us up too....",366606850,sandyB_real_10,NevenaBK,Fri Nov 02 13:54:23 +0000 2012,real,"Dear mainstream media , Sandy fucked us up too...",0
1,262962084801814528,Crazy pic taken in #NewYork today! #Hurricane ...,47332641,sandyA_fake_29,iChrisHarrison,Mon Oct 29 17:00:21 +0000 2012,fake,Crazy pic taken in #NewYork today ! #Hurricane...,1
2,264415086981038080,Hurricane A-rod #mlbmemes http://t.co/1cyx4OdM,63635369,sandyB_real_10,wiegman_tommy,Fri Nov 02 17:14:03 +0000 2012,real,Hurricane A - rod #mlbmemes <url>,0
3,591987644134887425,Nepal's historic Dharahara Tower collapses in ...,485782675,nepal_25,EliseArja,Sat Apr 25 15:30:27 +0000 2015,real,Nepal ' s historic Dharahara Tower collapses i...,0
4,263997194540032003,O furacão Sandy passou por Nova Iorque e destr...,354373541,sandyA_real_48,GALOUCURACORINT,Thu Nov 01 13:33:30 +0000 2012,real,Hurricane Sandy passed through New York and de...,0


,post_id,post_text,user_id,image_id(s),username,timestamp,label,cleaned_text,enc_label
0,263067887546281984,"Hurricane #Sandy, by @time http://t.co/ixAeykg1",112749098,sandyA_real_22,amelieamaral,Tue Oct 30 00:00:46 +0000 2012,real,"Hurricane #Sandy , by <user> <url>",0
1,263012006943158274,That shit cray.. #sandy http://t.co/PmI0RKvq,322326492,sandyA_fake_29,HoZe_B1,Mon Oct 29 20:18:43 +0000 2012,fake,That shit cray . . #sandy <url>,1
2,263218608790970369,The effects of hurricane sandy in Harlem New Y...,30071828,sandyA_real_20,AKidNameTraveee,Tue Oct 30 09:59:41 +0000 2012,real,The effects of hurricane sandy in Harlem New Y...,0
3,263098784236597248,O.O #Jesuuuu RT @Famosos_RD: Este tiburon no s...,180451067,sandyA_fake_32,edluis_11,Tue Oct 30 02:03:32 +0000 2012,fake,O. O #Jesuuuu RT <user>: This shark cannot com...,1
4,264017358262243328,"\""@TuiterHits: IMPRESIONANTE: Las inundaciones...",419835315,sandyA_fake_12,elenitacanon15,Thu Nov 01 14:53:38 +0000 2012,fake,"\ ""<user>: AWESOME: Flooding from Hurricane Sa...",1


,post_id,post_text,user_id,username,image_id,timestamp,label,cleaned_text,enc_label
0,665333038944002048,"Tristesse...😢🙏 \n#Bataclan sold out, musiciens...",5.547267e+08,Louise_Officiel,attacks_paris_1,Sat Nov 14 00:58:52 +0000 2015,fake,"Sadness. . . 😢 🙏 #Bataclan sold out, public mu...",1
1,665324167785410560,RT @Proyecto40: #ÚltimaHora Espectacular fotog...,2.827010e+09,MarinoCarril,attacks_paris_1,Sat Nov 14 00:23:37 +0000 2015,fake,<user>: # UltimaHora Spectacular photograph ta...,1
2,665333370205765632,RT @Javivi1976: #Bataclan esta noche antes de ...,3.776037e+08,MireiaMaroto,attacks_paris_1,Sat Nov 14 01:00:11 +0000 2015,fake,<user>: #Bataclan tonight before the attacks. ...,1
3,665326188735295489,RT @Pizzigatas: El hombre tiene que establecer...,1.102841e+08,maria_cabrera,attacks_paris_1,Sat Nov 14 00:31:39 +0000 2015,fake,<user>: Man has to establish an end to the war...,1
4,665389341141659648,🇫🇷 #Paris https://t.co/zjjRPC7USm,1.653118e+07,RaycMolina,attacks_paris_1,Sat Nov 14 04:42:35 +0000 2015,fake,🇫 🇷 #Paris <url>,1


(None, None, None)

In [ ]:
train.to_csv('/content/drive/MyDrive/mediaeval2016/train-ekphrasis.csv',index=False)
dev.to_csv('/content/drive/MyDrive/mediaeval2016/dev-ekphrasis.csv',index=False)
test.to_csv('/content/drive/MyDrive/mediaeval2016/test-ekphrasis.csv',index=False)

In [ ]:
test.iloc[739]['cleaned_text']

'<user> : الفنانة السويسرية ميلو مواريه تتعرى إحتجاجا على التحرش في ألمانيا " احترمونا ، نحن لسنا لعبة مجانية حتى ولو كنا عراة ! " https :// t …'

In [ ]:
translator.translate(test.iloc[2]['post_text']).text

'RT @Javivi1976: #Bataclan esta noche antes de los atentados. @playradiovlc https://t.co/ThfGdHtIjg'

In [ ]:
preprocessing(test.iloc[739]['post_text'])